In [3]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

def search_and_extract_details( license_number, license_type="Geotechnical Engineer"):
    """
    Searches for a specific license type and number on the DCA website,
    extracts 'More Detail' links, and saves them to a DataFrame.

    Args:
    license_type (str): The type of license to search for.
    license_number (str): The license number to search for.

    Returns:
    pd.DataFrame: A DataFrame containing the extracted 'More Detail' links.
    """

    # Initialize the WebDriver
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

    # Open the target website
    driver.get("https://search.dca.ca.gov/")

    # Increase wait time
    wait = WebDriverWait(driver, .1)

    try:
        # Find the "License Type" dropdown and select the desired option
        license_type_dropdown = wait.until(EC.presence_of_element_located((By.NAME, "licenseType")))
        for option in license_type_dropdown.find_elements(By.TAG_NAME, 'option'):
            if option.text == license_type:
                option.click()
                break

        # Find and fill in the "License Number" input field
        license_number_field = wait.until(EC.presence_of_element_located((By.NAME, "licenseNumber")))
        license_number_field.send_keys(license_number)

        # Submit the search
        search_button = wait.until(EC.element_to_be_clickable((By.ID, "srchSubmitHome")))
        search_button.click()

        # Wait for results to load
        time.sleep(5)

        # Parse the HTML content of the results page
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # Extract 'More Detail' links
        more_detail_buttons = soup.find_all('a', class_='button newTab', href=True)
        hrefs = [button['href'] for button in more_detail_buttons]
        df = pd.DataFrame(hrefs, columns=['More Detail Link'])

        return df

    finally:
        # Close the browser
        driver.quit()


In [5]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd
import time
import os

# Save DataFrame to CSV, appending if the file exists
def save_to_csv(df, filename, mode='a', header=False):
    df.to_csv(filename, mode=mode, header=header, index=False)

# Process a batch of URLs
def process_batch(urls):
    with ThreadPoolExecutor(max_workers=10) as executor:
        future_to_url = {executor.submit(search_and_extract_details, url): url for url in urls}
        results = []
        for future in as_completed(future_to_url):
            url = future_to_url[future]
            try:
                data = future.result()
                results.append(data)
            except Exception as e:
                print(f"Error processing {url}: {e}")
        return results

# Read missing licenses from CSV file
df_websites = pd.read_csv('missing_license_numbers_selected_geo.csv')

# Convert the specific column to a list, assuming the column name correctly identifies the URLs
urls_to_scrape = df_websites['Missing License Numbers'].tolist()

# Define batch size and delay
batch_size = 200
delay_between_batches = 20  # seconds

# Specify the output filename
output_filename = "details_geo_batch.csv"


# Process URLs in batches
for i in range(0, len(urls_to_scrape), batch_size):
    batch_urls = urls_to_scrape[i:i + batch_size]
    batch_results = process_batch(batch_urls)
    
    # Check if there are any non-empty DataFrames to concatenate
    non_empty_dfs = [df for df in batch_results if not df.empty]
    if non_empty_dfs:
        df_batch_details = pd.concat(non_empty_dfs, ignore_index=True)

        # Only proceed if df_batch_details is not empty (redundant check removed)
        df_batch_details.head()  # Display the first few rows if needed

        # Append batch results to CSV, creating the file if it does not exist yet
        file_exists = os.path.isfile(output_filename)
        df_batch_details.to_csv(output_filename, mode='a', header=not file_exists, index=False)
    
    # Remove processed URLs from the original DataFrame
    # Note: Ensure 'Missing License Numbers' or the relevant column name matches your DataFrame structure
    df_websites = df_websites[~df_websites['Missing License Numbers'].isin(batch_urls)]
    
    # Save the updated DataFrame back to the CSV to keep track of the progress
    df_websites.to_csv('missing_license_numbers_selected_geo.csv', index=False)
    
    # Delay before processing the next batch
    if i + batch_size < len(urls_to_scrape):
        time.sleep(delay_between_batches)

Error processing 1497: Message: 
Stacktrace:
	GetHandleVerifier [0x00B41673+52979]
	(No symbol) [0x00AC7961]
	(No symbol) [0x009ADD3D]
	(No symbol) [0x009E5FBB]
	(No symbol) [0x009E60FB]
	(No symbol) [0x00A1CF92]
	(No symbol) [0x00A04534]
	(No symbol) [0x00A1B3FE]
	(No symbol) [0x00A04286]
	(No symbol) [0x009DC063]
	(No symbol) [0x009DCECD]
	GetHandleVerifier [0x00E58D83+3294723]
	GetHandleVerifier [0x00E96CC2+3548482]
	GetHandleVerifier [0x00E91C9C+3527964]
	GetHandleVerifier [0x00BD870E+671630]
	(No symbol) [0x00AD1EB4]
	(No symbol) [0x00ACD808]
	(No symbol) [0x00ACD92D]
	(No symbol) [0x00ABF7E0]
	BaseThreadInitThunk [0x766EFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77DF7C5E+286]
	RtlGetAppContainerNamedObjectPath [0x77DF7C2E+238]

Error processing 1786: HTTPSConnectionPool(host='googlechromelabs.github.io', port=443): Read timed out. (read timeout=None)
